This is an analysis of the 2018 season Quarterback performance, speficially using the QB Rating. 

In [12]:
# imports
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200) # Display up to 120 columns of a dataframe

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from matplotlib import style



Import data from the great [ryurko / nflscrapR-data](https://github.com/ryurko/nflscrapR-data) . Your the best!

In [13]:
nfl = pd.read_csv('reg_pbp_2018.csv')
nfl.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,...,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,37,2018090600,PHI,ATL,ATL,away,PHI,PHI,35.0,2018-09-06,900.0,1800.0,3600.0,Half1,0,1,0,1,NaN,0.0,15:00:00,PHI 35,0,73,J.Elliott kicks 65 yards from PHI 35 to end zo...,kickoff,0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Set indicator for Week #s


In [14]:
import datetime
def game_weeks(df):
    """
    Create dataset indicate the game week. 
    """
    
    week_dict = {}
    first_game_date = df.min().date()
    game_date = first_game_date
    season_start_date = pd.to_datetime('2018-09-04 00:00:00').date()
    last_game_date = df.max().date()
    week_num = 1 # start week 1 and increment from there
    
    for i in range((last_game_date - first_game_date).days+1):
        if game_date <= season_start_date:
            week_dict[game_date] = 0
            game_date += datetime.timedelta(days=1)
        else:
            if game_date.weekday() == 1: # check if weekday is Tuesday.. if so reset week_number
                week_num += 1
            week_dict[game_date] = week_num
            game_date += datetime.timedelta(days=1)
    
    week_list = pd.Series(week_dict).to_frame()[0]
    
    return week_list

d = game_weeks(pd.to_datetime(nfl.game_date))
for o,j in enumerate(d.index):
    nfl.loc[nfl.game_date == str(j), 'week'] = d[o]

## Player: Passing Stats

In [26]:
qb = nfl[(nfl.play_type=='pass')].groupby(['passer_player_name']).agg({
    #'game_id':lambda x: x.nunique(), # unique counts for each game
    'complete_pass': sum,
    'pass_attempt':sum, 
    'yards_gained':sum,
    'touchdown':sum,
    'interception':sum,
    'first_down_pass':sum,
    'pass_attempt':sum,
    'sack':sum,
    #'success_play':sum
})


# net yards gained per pass attempts [PFR]
# yards gain includes detuctions on sack yards outcomes
qb['net_yrd_per_pass_att'] = qb.yards_gained / (qb.pass_attempt + qb.sack)

# completion %
qb['pct_cmp'] = round((qb.complete_pass / qb.pass_attempt), 1)    # receiving yards per reception 
qb['pct_td'] = round((qb.touchdown / qb.pass_attempt), 1)    # % of TDs when attempting a pass

#qb = qb[qb.pass_attempt >= 50]

## NFL Passer Rating Formula
The NFL passer rating formula includes four variables:
    - completions percentages
    - yards per attempt
    - touchdowns per attempt
    - interception per attempt

Each of these variables is scaled to a value between 0 and 2.375, with 
1.0 being statistically average (based on league data between 1960-1970).

Note: passing performance in 2017 as the league average rating was 88.6

Source: https://en.wikipedia.org/wiki/Passer_rating 

In [27]:
qb['cmp'] = ((qb.complete_pass / qb.pass_attempt) - .3) * 5           # completion percentage
qb['ypa'] = ((qb.yards_gained / qb.pass_attempt) - 3) * .25          # yards per attempt
qb['tdpa'] = (qb.touchdown / qb.pass_attempt) * 20                # touchdowns per attempt 
qb['intpa'] = 2.375 - ((qb.interception / qb.pass_attempt) * 25)     # interceptions per attempt 

def pass_rate_quality_check(rating):
    """
    If the result of any calculation is greater than 2.375, it is set to 2.375. 
    If the result is a negative number, it is set to zero.
    """
    if rating > 2.375:
        rating = 2.375

    elif rating < 0:
        rating = 0
    
    return rating


qb['cmp_q'] = qb.cmp.apply(lambda x: pass_rate_quality_check(x))
qb['ypa_q'] = qb.ypa.apply(lambda x: pass_rate_quality_check(x))
qb['tdpa_q'] = qb.tdpa.apply(lambda x: pass_rate_quality_check(x))
qb['intpa_q'] = qb.intpa.apply(lambda x: pass_rate_quality_check(x))

# Passer Rating

qb['pass_rate'] = ((qb['cmp_q'] + qb['ypa_q'] + qb['tdpa_q'] + qb['intpa_q']) / 6) * 100

qb.drop(['cmp_q','ypa_q','tdpa_q','intpa_q'], axis=1, inplace=True)

In [28]:
# Ranks
# minimum 50 passes
qb['qbr_rank'] = qb.groupby('passer_player_name')['pass_rate'].sum().rank(ascending=False)
qb.sort_values(['qbr_rank'], ascending = [True], inplace = True)
qb

,complete_pass,pass_attempt,yards_gained,touchdown,interception,first_down_pass,sack,net_yrd_per_pass_att,pct_cmp,pct_td,cmp,ypa,tdpa,intpa,pass_rate,qbr_rank
passer_player_name,,,,,,,,,,,,,,,,
R.Fitzpatrick,21.0,27.0,417,4.0,0.0,17.0,0.0,15.444444,0.8,0.1,2.388889,3.111111,2.962963,2.375000,158.333333,1.0
D.Brees,37.0,47.0,434,3.0,0.0,19.0,1.0,9.041667,0.8,0.1,2.436170,1.558511,1.276596,2.375000,126.418440,2.0
N.Agholor,1.0,1.0,15,0.0,0.0,1.0,0.0,15.000000,1.0,0.0,3.500000,3.000000,0.000000,2.375000,118.750000,3.0
J.Flacco,25.0,36.0,236,3.0,0.0,12.0,1.0,6.378378,0.7,0.1,1.972222,0.888889,1.666667,2.375000,115.046296,4.0
A.Dalton,21.0,30.0,229,2.0,1.0,11.0,2.0,7.156250,0.7,0.1,2.000000,1.158333,1.333333,1.541667,100.555556,5.0
T.Brady,26.0,41.0,267,3.0,1.0,16.0,2.0,6.209302,0.6,0.1,1.670732,0.878049,1.463415,1.765244,96.290650,6.0
A.Luck,39.0,55.0,305,3.0,1.0,17.0,2.0,5.350877,0.7,0.1,2.045455,0.636364,1.090909,1.920455,94.886364,7.0
K.Cousins,20.0,39.0,227,2.0,0.0,15.0,3.0,5.404762,0.5,0.1,1.064103,0.705128,1.025641,2.375000,86.164530,8.0
R.Tannehill,20.0,29.0,222,2.0,2.0,9.0,1.0,7.400000,0.7,0.1,1.948276,1.163793,1.379310,0.650862,85.704023,9.0
